In [ ]:
key = "<your groq key>"

In [15]:
from langchain_groq import ChatGroq

In [16]:
llm = ChatGroq(
        temperature=0,
        groq_api_key = key
        #model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("Who is the PM of India?")

response.content

'The current Prime Minister of India is Narendra Modi. He has been in office since May 2014. He is a member of the Bharatiya Janata Party (BJP) and is also a member of parliament from the Varanasi constituency. Before serving as Prime Minister, Modi was the Chief Minister of Gujarat from 2001 to 2014. He is known for his economic reforms and nationalist policies.'

In [1]:
import chromadb

In [2]:
client = chromadb.Client()

In [3]:
collection = client.create_collection(name="my_collection")

In [4]:
collection.add(
    documents=["new delhi", "Jabalpur", "Mumbai"],
    ids=["id1", "id2", "id3"])

C:\Users\SRMAP\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|█████| 79.3M/79.3M [00:57<00:00, 1.45MiB/s]


In [11]:
collection.query(
    query_texts=[' air polution'],
    n_results=2
)

{'ids': [['id3', 'id1']],
 'distances': [[1.599498987197876, 1.71663498878479]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['Mumbai', 'new delhi']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [12]:
from langchain_community.document_loaders import WebBaseLoader

In [29]:
loader = WebBaseLoader("https://jobs.nike.com/job/R-36945?from=job%20search%20funnel")

page_data = loader.load().pop().page_content

print(page_data)

Apply for Lead Cyber Defense Incident Responder

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Cyber Defense Incident ResponderShanghai, China MainlandBecome a Part of t

In [30]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### Scraped Text from Website:
    {page_data}
    
    ### Instructions:
    This data is coming from a career's page of a website.
    Your Job is to extract the Job postings and return them in the JSON format containing the following keys:
    'role', 'experience', 'skills' and 'description'.
    Only return the valid JSON.
    
    ###VALID JSON (NO PREAMBLE):  
    
    """)

chain_extract = prompt_extract | llm

response = chain_extract.invoke(input={'page_data':page_data})

In [31]:
print(response.content)

{
"role": "Lead Cyber Defense Incident Responder",
"experience": "Minimum five years of security monitoring experience and incident response activities; preferably within a professional services firm or similar environment",
"skills": ["Solid knowledge of information security principles and practice", "Incident response and crisis management", "Network, desktop and server technologies", "Cloud security", "Log analysis", "Proficiency with Windows, Mac OSX and Linux", "Application security and dev-sec-ops", "Coding/scripting experience (PowerShell, Python, Ruby, Java, VB Scripting etc.)", "Malware analysis, virus exploitation and mitigation techniques experience", "Quick learning abilities", "Problem-solving skills", "Knowledge of ITIL best practices and process improvements"],
"description": "The Lead Cyber Defense Incident Responder will be responsible for identifying, containing and eradicating cyber security incidents. This position uses a wide range of tools and experience to perfor

In [34]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

json_data = json_parser.parse(response.content)

In [36]:
json_data

{'role': 'Lead Cyber Defense Incident Responder',
 'experience': 'Minimum five years of security monitoring experience and incident response activities; preferably within a professional services firm or similar environment',
 'skills': ['Solid knowledge of information security principles and practice',
  'Incident response and crisis management',
  'Network, desktop and server technologies',
  'Cloud security',
  'Log analysis',
  'Proficiency with Windows, Mac OSX and Linux',
  'Application security and dev-sec-ops',
  'Coding/scripting experience (PowerShell, Python, Ruby, Java, VB Scripting etc.)',
  'Malware analysis, virus exploitation and mitigation techniques experience',
  'Quick learning abilities',
  'Problem-solving skills',
  'Knowledge of ITIL best practices and process improvements'],
 'description': 'The Lead Cyber Defense Incident Responder will be responsible for identifying, containing and eradicating cyber security incidents. This position uses a wide range of tools 

In [38]:
import pandas as pd

portfolios = pd.read_csv("my_portfolio.csv")

In [39]:
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolios")


In [44]:
import uuid
if not collection.count():
    for _, row in portfolios.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links": row["Links"]},
                      ids=[str(uuid.uuid4())])

In [51]:
links = collection.query(
    query_texts=json_data['skills'],
    n_results=2
).get('metadatas', [])

In [52]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(json_data), "link_list": links})
print(res.content)

Subject: Experienced Lead Cyber Defense Incident Responder Services at AtliQ

Dear Hiring Manager,

I came across the Lead Cyber Defense Incident Responder job posting and believe AtliQ can provide the expertise and skills required for this role. Our team has extensive experience in security monitoring, incident response, and cloud security, with a strong focus on application security and Unix/Linux operating systems.

AtliQ's expertise includes:

1. Solid knowledge of information security principles and practices
2. Incident response and crisis management
3. Network, desktop, and server technologies
4. Cloud security and log analysis
5. Proficiency with Windows, Mac OSX, and Linux
6. Application security and dev-sec-ops
7. Coding/scripting experience (Python, PowerShell, Ruby, Java, VB Scripting etc.)
8. Malware analysis, virus exploitation, and mitigation techniques
9. Quick learning abilities and problem-solving skills

Our team has a proven track record of identifying, containing, 